In [4]:
from langchain.prompts import PromptTemplate
from langchain.prompts.chat import ChatPromptTemplate, HumanMessagePromptTemplate, AIMessagePromptTemplate, SystemMessagePromptTemplate

In [5]:
import json
import os

data_path = os.path.join(".",
                         "artifacts",
                         "gpt-4_Chat_Tables_Chunk-size=2500_Overlap=500_Doc=10_Max-token=6500_Stuff_22-08-2023-23-30-50")

with open(os.path.join(data_path, "result1.json"), "r") as f:
    data = json.load(f)

In [12]:
indices = [0,1,3,6,8,9,10,11,20,21,24,26,27,28,29]
len(indices)

15

In [13]:
ez_data = []
for idx in indices:
    ez_data.append(data["test_cases"][idx])

In [27]:
ez_data

[{'question': '40 year old male with newly diagnosed moderate UC and articular extraintestinal manifestations\n',
  'answer': 'Based on the patient profile, the patient is newly diagnosed with moderate Ulcerative Colitis (UC) and presents with articular extraintestinal manifestations. No prior response to Infliximab or failure to Anti-TNF agents, Vedolizumab is mentioned. The patient is 40 years old. \n\nHere are the top two choices for biological drugs considering the patient\'s profile:\n\n1. Drug: Infliximab\n - Advantages: Infliximab has proven efficacy in inducing clinical remission and endoscopic improvement in biologic-naive patients with moderate-severe UC. Infliximab also has been effective in managing extraintestinal manifestations such as articular symptoms. \n - Disadvantages: Infliximab can cause adverse effects including increased risk of infection and potential for immunogenicity, leading to loss of response. It requires intravenous administration which might be inconven

In [29]:
data = {}
data["prompt"] = QuestionAnsweringWithIndexSearchExperiment.convert_prompt_to_string(PROMPT_TEMPLATE)
data["test_cases"] = ez_data

In [31]:
with open(os.path.join(data_path, "result1.json"), "w") as f:
    json.dump(data, f)

In [25]:
data

In [20]:
system_prompt = """
Make reference to the context given to assess the scenario. If you do not know the answer. just say that "I don't know", don't try to make up an answer.
You are a physician assistant giving advice on treatment for moderate to severe ulcerative colitis (UC).

ANALYSE the given patient profile based on given query based on one of the following criteria:
- Whether treated patient is new patient or patient under maintenance
- Prior response to Infliximab
- Prior failure to Anti-TNF agents
- Prior failure to Vedolizumab
- Age
- Pregnancy
- Extraintestinale manifestations
- Pouchitis

FINALLY RETURN up to 2 TOP choices of biological drugs given patient profile. Explain the PROS and CONS of the 2 choices.
Output your answer as a list of JSON objects with keys: drug_name, advantages, disadvantages.

{summaries}
"""

PROMPT_TEMPLATE = ChatPromptTemplate.from_messages(
    [
        SystemMessagePromptTemplate.from_template(
            system_prompt, input_variables=["summaries"]
        ),
        HumanMessagePromptTemplate.from_template("{question}"),
    ]
)

In [34]:
from exp import QuestionAnsweringWithIndexSearchExperiment

no_returned_docs = 10

experiment = QuestionAnsweringWithIndexSearchExperiment(
    prompt_template=PROMPT_TEMPLATE,
    vector_store=os.path.join("./data/emb_store/uc/faiss/text-embedding-ada-002/v8-add-tables_2500_500"),
    llm_type="gpt-4",
    emb="text-embedding-ada-002",
    temperature=0.7,
    max_tokens=1024,
    gt=os.path.join(".", "data", "queries", "uc_easy_gt.csv"),
    verbose=True,
    max_tokens_limit=6500,
    k=10,
    reduce_k_below_max_tokens=True,
)

2023-08-23 02:26:06,857:INFO: Successfully loaded existing vectorstore from local storage


In [32]:
experiment.load_json(os.path.join(data_path, "result1.json"))

2023-08-23 02:25:42,229:INFO: Json file loaded successfully into Experiment instance.


In [35]:
experiment.load_json(os.path.join(data_path, "result1.json"))
experiment.write_csv(os.path.join(data_path, "result1.csv"),
                     num_docs=10)

2023-08-23 02:26:12,003:INFO: Json file loaded successfully into Experiment instance.
